In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key


In [2]:
# Store the CSV you saved created in part one into a DataFrame & read the CSV
city_data_df = pd.read_csv("Data/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain(inches),Snow(inches)
0,0,Khatanga,RU,71.97,102.50,35.73,95,100,14.09,light snow,0.0,0.0
1,1,Alyangula,AU,-13.85,136.42,62.60,100,100,10.13,overcast clouds,0.0,0.0
2,2,Ndioum,SN,16.51,-14.65,111.20,14,70,4.70,broken clouds,0.0,0.0
3,3,Bati,ET,11.18,40.02,74.86,40,21,4.05,few clouds,0.0,0.0
4,4,Tacuati,PY,-23.45,-56.58,80.73,32,0,3.98,clear sky,0.0,0.0


In [3]:
# Ask the customer to add a minimum temp value,maximum temperature value, Rain or snow
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_preference = (input("Do you want it to be raining?(yes/no)"))
snow_preference = (input("Do you want it to be snowing?(yes/no)"))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining?(yes/no)no
Do you want it to be snowing?(yes/no)no


In [4]:
# Filter the dataset to find the cities that fit the criteria.

if rain_preference == 'no':
   preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp) & (city_data_df['Rain(inches)']== 0)]
elif rain_preference == 'yes':
   preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp) & (city_data_df['Rain(inches)']!= 0)] 
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain(inches),Snow(inches)
4,4,Tacuati,PY,-23.45,-56.58,80.73,32,0,3.98,clear sky,0.0,0.0
5,5,Luanda,AO,-8.84,13.23,77.00,88,40,5.82,scattered clouds,0.0,0.0
6,6,San Cristobal,VE,7.77,-72.22,86.00,62,40,10.29,scattered clouds,0.0,0.0
8,8,Kahului,US,20.89,-156.47,80.60,57,20,14.99,few clouds,0.0,0.0
10,10,Biak,ID,-0.91,122.88,78.22,79,73,5.01,broken clouds,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
496,496,Sakete,BJ,6.74,2.66,76.39,94,100,9.82,heavy intensity rain,0.0,0.0
499,499,Kavieng,PG,-2.57,150.80,81.84,76,12,4.21,light rain,0.0,0.0
501,501,Cam Ranh,VN,11.92,109.16,84.20,83,20,1.12,few clouds,0.0,0.0
503,503,Saint-Georges,GD,12.06,-61.75,87.80,66,20,18.34,few clouds,0.0,0.0


In [5]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Tacuati,PY,80.73,clear sky,-23.45,-56.58,
5,Luanda,AO,77.00,scattered clouds,-8.84,13.23,
6,San Cristobal,VE,86.00,scattered clouds,7.77,-72.22,
8,Kahului,US,80.60,few clouds,20.89,-156.47,
10,Biak,ID,78.22,broken clouds,-0.91,122.88,
...,...,...,...,...,...,...,...
496,Sakete,BJ,76.39,heavy intensity rain,6.74,2.66,
499,Kavieng,PG,81.84,light rain,-2.57,150.80,
501,Cam Ranh,VN,84.20,few clouds,11.92,109.16,
503,Saint-Georges,GD,87.80,few clouds,12.06,-61.75,


In [6]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

In [7]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    #hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    # Grab the first hotel from the results and store the name.

    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [8]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Tacuati,PY,80.73,clear sky,-23.45,-56.58,
5,Luanda,AO,77.00,scattered clouds,-8.84,13.23,EPIC SANA Luanda
6,San Cristobal,VE,86.00,scattered clouds,7.77,-72.22,Pirineos
8,Kahului,US,80.60,few clouds,20.89,-156.47,Maui Seaside Hotel
10,Biak,ID,78.22,broken clouds,-0.91,122.88,Kost Ariz
...,...,...,...,...,...,...,...
496,Sakete,BJ,76.39,heavy intensity rain,6.74,2.66,
499,Kavieng,PG,81.84,light rain,-2.57,150.80,Nusa Island Retreat
501,Cam Ranh,VN,84.20,few clouds,11.92,109.16,OYO 598 Peony Hotel
503,Saint-Georges,GD,87.80,few clouds,12.06,-61.75,Radisson Grenada Beach Resort


In [9]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)


In [10]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, 
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [13]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [14]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
# Create the output file (CSV).
output_data_file = "Data/WeatherPy_vacation.csv"

# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index=False)
